In [1]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
from pdf2image import convert_from_path
import datetime
import os
now = datetime.date.today()

# カレントディレクトリの全画像をループする
for pdfname in os.listdir('./mergedTRF/'):# ❶
    ext = os.path.splitext(pdfname)
    #if ext == '.pdf':
    images = convert_from_path('./mergedTRF/'+pdfname)
    os.makedirs('./trf/'+str(now), exist_ok=True)
    i = 0
    for image in images:
        image.save('./trf/'+str(now) +'/trf_' + str(pdfname)+'_'+str(i)+'.png', 'png')
        i += 1
       

In [2]:
checkmark = [
    (1175,1225,95,145),#US 1
    (1185,1235,260,310),#LMP 2
    (1185,1235,455,505), #IVF 
    (1185,1235,645,695),#PE 
    (1280,1330,65,115), #自然妊娠 
    (1330,1380,65,115), #新鮮胚利用
    (1380,1430,60,110), #融解胚利用
 (1425,1475,60,110), #提供卵子利用
    # 4 胎児数
(1725,1775,80,130), #単胎 10
(1730,1780,215,265), #双胎
    # 5 人種
 (1860,1910,55,105), # Japanese
 (1905,1955,55,105), # Caucasian
(1945,1995,55,105),# African American
(1990,2040,55,105), # Other
    # IDDM
 (995,1045,940,990), # OIDDM No
 (995,1045,1120,1170), # OIDDM Yes
    #T21
 (1145,1195,940,990), # DS No
(1145,1195,1120,1170), # DS Yes
    #NTD
 (1585,1635,935,985), # DS No 20
 (1665,1715,935,985), # DS yes
    #NTD relation
 (1785,1835,1070,1120), # Patient herself
 (1835,1885,1070,1120), # Husbond of Patient 
(1880,1930,1070,1120), # Mother of
 (1925,1975,1070,1120), # siblings 25
 (1970,2020,1070,1120), # previous child 26
    ]

In [3]:
#PythonでOCRがんばる(1) hhttp://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916040943/
#tesseractでOCR〜数字だけでも〜 http://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916052946/
# p87 17.3.2 ステップ2：全ファイルをループして画像を開く https://www.oreilly.co.jp/pub/9784873117782/auto-sample.pdf

from PIL import Image
import cv2
import sys,os
import pyocr
import pyocr.builders
import csv
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

%matplotlib inline
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

labels = ["Error ", "N ", "Y "]
os.makedirs('./results/checkmark_results/'+ str(now), exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Error', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/N', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Y', exist_ok = True)
os.makedirs('./csv/'+ str(now), exist_ok = True)

os.makedirs('./results/PtID_results/'+ str(now), exist_ok = True)

# モデルを読込 --- (*1)
model = load_model('CNN-mk-model.h5')
# 重みデータを読込 --- (*2)
model.load_weights('CNN-mk-weight.h5')

# カレントディレクトリの全画像をループする
for filename in os.listdir('./trf/'+ str(now)+'/'):# ❶
    ext = os.path.splitext(filename)
    if ext == '.png' or u'.jpeg' or u'.jpg':
        img = Image.open('./trf/'+ str(now)+'/' + filename)  # ❹
        img = img.crop((800,129,1050,190))
        img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(filename))

        tools = pyocr.get_available_tools()
        if len(tools) == 0:
            print("No OCR tool found")
            sys.exit(1)
        # The tools are returned in the recommended order of usage
        tool = tools[0] 
        txt = tool.image_to_string(
            Image.open('./results/PtID_results/'+ str(now)+'/' +  "PtID_" + str(filename)),
            lang="eng",
            builder=pyocr.tesseract.DigitBuilder(),
            )
        #img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(now_time)+'_'+ str(filename))
        path_w = './csv/'+ str(now)+'/'+str(now)+'_'+str(now_time)+'.csv'
        with open(path_w, mode='a') as f:
            f.write(txt+'\t')
            txt.replace('-','') 
            f.write(filename+'\t')
            
    for j, pt in enumerate(checkmark):
        x,y,w,h = pt
        img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
        img = img[x:y,w:h]
                
        #データ予測する
        res = model.predict(np.array([img]), batch_size=32)
        idx = res.argmax()
        #cv2.imwrite('./'+ str(now) + '/'+str(labels[idx]) + str(now)+ 'mk_'+ str(j)+'_'+ filename, img)
        cv2.imwrite('./results/checkmark_results/' + str(now) + '/'+ str(labels[idx].strip())+\
                        '/' + str(now)+ 'mk_'+ str(j)+'_'+ filename, img)
        j += 1
        #path_w = './csv/'+str(now)+'/checkmark_'+str(now)+'_'+ str(now_time)+'.csv'
        with open(path_w, mode='a') as f:
            f.write(str(labels[idx])+'\t')
    with open(path_w, mode='a') as f:
        f.write('\n')
    with open(path_w) as f:
        print(f.read())    
import shutil
shutil.move('./results/PtID_results/' + str(now)+'/','./results/PtID_results/' + str(now_time)+'/')
shutil.move('./trf/'+str(now)+'/','./finished_trf/'+str(now)+'_'+str(now_time)+'/')

Using TensorFlow backend.


2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

...

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

2502525	trf_41.pdf_5.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
20033140	trf_21.pdf_0.png	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
Fns
2799221	trf_11.pdf_1.png	Y 	Error 	Error 	Error 	Y 	N 	Error 	N 	Y 	N 	Y 	N 	Error 	N 	Y 	Error 	Y 	Error 	Y 	N 	N 	N 	N 	N 	N 	
2006745	trf_61-67.pdf_6.png	Error 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Y 	Error 	Error 	Error 	Error 	Y 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Error 	
2501318	trf_41.pdf_1.png	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Y 	Y 	Error 	Error 	Y 	Error 	Y 	Error 	Y 	Error 	Error 	Error 	Error 	Error 	Y 	
2725601 -	trf_21.pdf_7.png	Y 	N 	N 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	Y 	N 	N 	N 	N 	N 	N 	
2745199 _	trf_1.pdf_2.png	Y 	N 	N 	Error 	Y 	N 	N 	N 	Y 	N 	Y 	N 	N 	N 	Y 	N 	Y 	N 	

'./finished_trf/2019-08-06_11-53-18/'